# Multi-Channel Synchrotron Data Visualization

This notebook demonstrates visualization techniques for synchrotron data, including:
- XRF elemental maps (multi-channel 2D images)
- Tomography projections and sinograms
- Reconstructed tomography slices

**Prerequisites**: `pip install h5py numpy matplotlib`

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["figure.dpi"] = 120
plt.rcParams["font.size"] = 10

# Update these paths to your local data files
XRF_FILE = "xrf_scan.h5"
TOMO_FILE = "tomo_scan.h5"

In [ ]:
# XRF: Load and display elemental maps as a grid

with h5py.File(XRF_FILE, "r") as f:
    maps = f["MAPS/XRF_Analyzed/Fitted/Counts_Per_Sec"][:]
    names = [n.decode() for n in f["MAPS/XRF_Analyzed/Channel_Names"][:]]
    x_axis = f["MAPS/Scan/x_axis"][:]
    y_axis = f["MAPS/Scan/y_axis"][:]

nelem = len(names)
ncols = 5
nrows_grid = int(np.ceil(nelem / ncols))

fig, axes = plt.subplots(nrows_grid, ncols, figsize=(4 * ncols, 3.5 * nrows_grid))
axes = axes.ravel()

extent = [x_axis[0], x_axis[-1], y_axis[0], y_axis[-1]]

for i in range(nelem):
    ax = axes[i]
    data = maps[i]
    vmin = np.percentile(data[data > 0], 2) if (data > 0).any() else 0
    vmax = np.percentile(data, 99)
    im = ax.imshow(data, cmap="inferno", origin="lower", extent=extent,
                   vmin=vmin, vmax=vmax)
    ax.set_title(names[i], fontweight="bold")
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)

# Hide unused axes
for i in range(nelem, len(axes)):
    axes[i].set_visible(False)

fig.supxlabel("X (um)")
fig.supylabel("Y (um)")
plt.tight_layout()
plt.savefig("xrf_all_elements.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# XRF: RGB composite visualization

def make_rgb(maps, names, r_elem, g_elem, b_elem, clip_pct=99):
    """Create RGB composite from three elemental channels."""
    rgb = np.zeros((*maps[0].shape, 3), dtype=float)
    for ch, elem in enumerate([r_elem, g_elem, b_elem]):
        idx = names.index(elem)
        arr = maps[idx].astype(float)
        lo = np.percentile(arr, 1)
        hi = np.percentile(arr, clip_pct)
        rgb[:, :, ch] = np.clip((arr - lo) / (hi - lo + 1e-10), 0, 1)
    return rgb

# Choose element combinations relevant to your sample
# Common for environmental science: Fe, Ca, Zn
combos = [
    ("Fe", "Ca", "Zn"),
    ("Fe", "Mn", "Cu"),
    ("S", "K", "P"),
]

# Filter to only include combos where all elements exist
valid_combos = [c for c in combos if all(e in names for e in c)]

if valid_combos:
    fig, axes = plt.subplots(1, len(valid_combos),
                             figsize=(6 * len(valid_combos), 5))
    if len(valid_combos) == 1:
        axes = [axes]
    for ax, (r, g, b) in zip(axes, valid_combos):
        rgb = make_rgb(maps, names, r, g, b)
        ax.imshow(rgb, origin="lower", extent=extent)
        ax.set_title(f"R={r}  G={g}  B={b}")
        ax.set_xlabel("X (um)")
        ax.set_ylabel("Y (um)")
    plt.tight_layout()
    plt.savefig("xrf_rgb_composites.png", dpi=150, bbox_inches="tight")
    plt.show()
else:
    print("Update element names above to match your dataset.")

In [ ]:
# Tomography: Projections at selected angles and sinogram view

with h5py.File(TOMO_FILE, "r") as f:
    proj_dset = f["/exchange/data"]
    theta = f["/exchange/theta"][:]
    nproj, nrow, ncol = proj_dset.shape
    print(f"Projections: {nproj}, Image: {nrow} x {ncol}")
    print(f"Theta: {np.degrees(theta[0]):.1f} to {np.degrees(theta[-1]):.1f} deg")

    # Select 4 evenly spaced projections
    indices = np.linspace(0, nproj - 1, 4, dtype=int)
    fig, axes = plt.subplots(2, 4, figsize=(18, 8))

    # Top row: projections
    for i, idx in enumerate(indices):
        proj = proj_dset[idx]
        axes[0, i].imshow(proj, cmap="gray", origin="lower")
        axes[0, i].set_title(f"Proj #{idx} ({np.degrees(theta[idx]):.1f} deg)")

    # Bottom row: sinograms at different heights
    sino_rows = [nrow // 4, nrow // 2, 3 * nrow // 4, nrow - 10]
    for i, row in enumerate(sino_rows):
        sino = proj_dset[:, row, :]
        axes[1, i].imshow(sino, cmap="gray", aspect="auto", origin="lower")
        axes[1, i].set_title(f"Sinogram row={row}")
        axes[1, i].set_xlabel("Detector Column")
        axes[1, i].set_ylabel("Projection Index")

plt.tight_layout()
plt.savefig("tomo_projections_sinograms.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Tomography: Flat-field, dark-field, and normalized projection comparison

with h5py.File(TOMO_FILE, "r") as f:
    proj_0 = f["/exchange/data"][0].astype(float)
    flat = np.mean(f["/exchange/data_white"][:].astype(float), axis=0)
    dark = np.mean(f["/exchange/data_dark"][:].astype(float), axis=0)

# Normalize
normalized = (proj_0 - dark) / np.clip(flat - dark, 1, None)
absorption = -np.log(np.clip(normalized, 1e-6, None))

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

titles = ["Raw Projection", "Mean Flat Field", "Normalized (I/I0)", "Absorption (-log)"]
images = [proj_0, flat, normalized, absorption]
cmaps = ["gray", "gray", "gray", "viridis"]

for ax, title, img, cmap in zip(axes, titles, images, cmaps):
    vmin = np.percentile(img, 1)
    vmax = np.percentile(img, 99)
    im = ax.imshow(img, cmap=cmap, origin="lower", vmin=vmin, vmax=vmax)
    ax.set_title(title)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)

plt.tight_layout()
plt.savefig("tomo_normalization_steps.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Interactive intensity profile along a line

# XRF: line profile across an elemental map
if len(names) > 0:
    # Pick the first element with nonzero signal
    elem_idx = 0
    for i, n in enumerate(names):
        if maps[i].max() > 0:
            elem_idx = i
            break
    
    elem_map = maps[elem_idx]
    mid_row = elem_map.shape[0] // 2
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Map with line overlay
    axes[0].imshow(elem_map, cmap="inferno", origin="lower", extent=extent)
    axes[0].axhline(y_axis[mid_row], color="cyan", lw=1.5, ls="--")
    axes[0].set_title(f"{names[elem_idx]} Map")
    axes[0].set_xlabel("X (um)")
    axes[0].set_ylabel("Y (um)")
    
    # Line profile
    profile = elem_map[mid_row, :]
    axes[1].plot(x_axis, profile, "b-", lw=0.8)
    axes[1].fill_between(x_axis, 0, profile, alpha=0.2)
    axes[1].set_xlabel("X (um)")
    axes[1].set_ylabel(f"{names[elem_idx]} (counts/sec)")
    axes[1].set_title(f"Line Profile at Y = {y_axis[mid_row]:.1f} um")
    
    plt.tight_layout()
    plt.show()

## Summary

Visualization techniques demonstrated:

| Technique | Use Case |
|-----------|----------|
| Elemental map grid | Survey all XRF channels at a glance |
| RGB composites | Reveal spatial correlations between 3 elements |
| Projection gallery | Inspect tomography data at multiple angles |
| Sinogram display | Diagnose rotation center errors and ring artifacts |
| Normalization comparison | Verify flat/dark correction quality |
| Line profiles | Quantitative 1D cross-section analysis |

**Tips**:
- Use `vmin`/`vmax` with percentile clipping to handle outliers
- Log normalization (`LogNorm`) is useful for data spanning many orders of magnitude
- Save figures at 150+ DPI for publication quality
- Consider `silx view` for interactive GUI-based HDF5 exploration